In [28]:
import fiona
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, box, mapping
from shapely import speedups
from pathlib import Path
import os
import gc
from zipfile import ZipFile, ZIP_DEFLATED

speedups.enable()


In [3]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR = Path(os.getcwd())
print(WORK_DIR)
BASE_DIR = f'{WORK_DIR.parents[3]}/work/datasets'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets', f'{BASE_DIR} is not the correct directory'

mangrove_extent_path = Path(f'{BASE_DIR}/raw/extent-layer-creation/gmw_v3_fnl_mjr_v314.gpkg')
layers = fiona.listlayers(mangrove_extent_path)
extent_masks_path = Path(f'{BASE_DIR}/raw/extent-layer-creation')

/home/jovyan/work/notebooks/Lab/data_processing


In [ ]:
def create_mask(dataset: Path, layer: str):
    extent_mask = gpd.read_file(mangrove_extent_path, layer=layer
                ).convex_hull.to_crs('epsg:3410').buffer(500)
    
    gpd.GeoDataFrame({"geometry": extent_mask.clip(
        gpd.GeoSeries({"geometry": box(-180,-50, 180, 40)}, crs='EPSG:4326'
        ).to_crs('EPSG:3410')
    ).to_crs('EPSG:4326').unary_union}, 
                     crs='EPSG:4326'
    ).to_file(f'{extent_masks_path}/{layer}-convex-bufered.shp')

In [6]:
for layer_name in layers:
    print(layer_name)
    create_mask(mangrove_extent_path, layer_name)

del layers
gc.collect()

mng_mjr_1996
mng_mjr_2007
mng_mjr_2008
mng_mjr_2009
mng_mjr_2010
mng_mjr_2015
mng_mjr_2016
mng_mjr_2017
mng_mjr_2018
mng_mjr_2019
mng_mjr_2020


In [7]:
# Combine all the masks into a single mask
simp_masks = [gpd.read_file(mask) 
    for mask in extent_masks_path.glob('*-convex-bufered.shp')]
geometry = gpd.pd.concat(simp_masks).unary_union
gpd.GeoDataFrame({"geometry": geometry}, 
                crs='EPSG:4326').to_file(
                    f'{extent_masks_path}/merged-convex-bufered.shp')

In [11]:
# This creates a shapefile for the mangrove coverage from 
# the extent that can be later use for masking the vector data like wdpa or gadm/eez

!mapshaper-xl 16gb -i '{extent_masks_path}/merged-convex-bufered.shp' snap \
    -simplify 0.10 planar keep-shapes \
    -o  '{extent_masks_path}/merged-convex-bufered-simp-10.shp' format=shapefile force

Allocating 16 GB of heap memory
[i] Snapped 267039 points
[simplify] Repaired 5 intersections
[o] Wrote /home/jovyan/work/datasets/raw/extent-layer-creation/merged-convex-bufered-simp-10.shp
[o] Wrote /home/jovyan/work/datasets/raw/extent-layer-creation/merged-convex-bufered-simp-10.shx
[o] Wrote /home/jovyan/work/datasets/raw/extent-layer-creation/merged-convex-bufered-simp-10.dbf
[o] Wrote /home/jovyan/work/datasets/raw/extent-layer-creation/merged-convex-bufered-simp-10.prj


In [37]:
with ZipFile(f'{extent_masks_path}/merged-convex-bufered-simp-10.zip', "w", ZIP_DEFLATED, compresslevel=9) as archive:
    for file in extent_masks_path.glob('*-convex-bufered-simp-*'):
        if file.suffix != '.zip':
            archive.write(file, arcname=file.relative_to(extent_masks_path))

In [19]:
for file in extent_masks_path.glob('*-convex-bufered.*'):
    os.remove(file)